# **Category Cleanup**

## Importing packages

In [3]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher, get_close_matches

import warnings
warnings.filterwarnings("ignore")

In [2]:
# loading the product list
iprocure_prods_df = pd.read_excel('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v2/product_list.xlsx')
cols = iprocure_prods_df.columns[:15]
iprocure_prods_df = iprocure_prods_df[cols]
iprocure_prods_df.drop(['Unit Cost', 'Unnamed: 10', 'Active'], axis = 1, inplace = True)

# dropping product_name duplicates
iprocure_prods_df = iprocure_prods_df.drop_duplicates(subset=['Product Name'], keep='first').reset_index(drop=True)
# iprocure_prods_df['combined_product_name'] = iprocure_prods_df[['Product Name', 'Manufacturer']].apply(lambda x: '; '.join(x.fillna('').astype(str)) if x.notna().any() else '', axis = 1)

pd.set_option('display.max_columns',None)
iprocure_prods_df.head()

,Category,Product Name,Type,Weight,Carton Size,Distributor,Product Code,Product Id,Distributor Type,Manufacturer,Industry,Sub category
0,Minerals and Supplements,Aviboost Aqua Block,1kg,1.0,1,Vital Animal Health,ANP001A,2672,1,Vital Animal Health,Agriculture,NaN
1,Minerals and Supplements,Aviboost CL-X Blue,1lt,1.0,1,Vital Animal Health,ANP061A,2674,1,Vital Animal Health,Agriculture,NaN
2,Minerals and Supplements,Aviboost Nutri Block,1kg,1.0,1,Vital Animal Health,ANP062A,2676,1,Vital Animal Health,Agriculture,NaN
3,Minerals and Supplements,Aviboost Spectrum,1lt,1.0,1,Vital Animal Health,ANP063A,2677,1,Vital Animal Health,Agriculture,NaN
4,Minerals and Supplements,Aviboost Poultry Tonic,1lt,1.0,1,Vital Animal Health,ANP003A,2678,1,Vital Animal Health,Agriculture,NaN


In [4]:
# size of dataset
iprocure_prods_df.shape

(4771, 12)

In [5]:
# loading category data
category_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v2/dirty_category_data.csv')

# dropping duplicates and renaning col
category_df = category_df.drop_duplicates(subset=['correct_product_match'], keep='first').reset_index(drop=True)
category_df.rename(columns={'correct_product_match': 'product_name'},inplace=True)
category_df.head()

,product_name,category_name,sub_category
0,gradometer 480 ec,Local,NaN
1,magazine east african,locall,NaN
2,vectoclor,VETERINARY PRODUCTS,NaN
3,chastity perfume men,Perfume,NaN
4,pannar 15m sunflower,seeds,NaN


In [6]:
# size of dataset
category_df.shape

(22595, 3)

In [7]:
category_df.isna().sum()

product_name         0
category_name        6
sub_category     22396
dtype: int64

In [8]:
# changing product name and category columns to lowercase
iprocure_prods_df[['Category', 'Product Name']] = iprocure_prods_df[['Category', 'Product Name']].applymap(lambda x: str(x).lower())
category_df[['category_name', 'product_name']] = category_df[['category_name', 'product_name']].applymap(lambda x: str(x).lower())

# renaming product name
iprocure_prods_df.rename(columns={'Product Name': 'product_name'}, inplace=True)
iprocure_prods_df.head()

,Category,product_name,Type,Weight,Carton Size,Distributor,Product Code,Product Id,Distributor Type,Manufacturer,Industry,Sub category
0,minerals and supplements,aviboost aqua block,1kg,1.0,1,Vital Animal Health,ANP001A,2672,1,Vital Animal Health,Agriculture,NaN
1,minerals and supplements,aviboost cl-x blue,1lt,1.0,1,Vital Animal Health,ANP061A,2674,1,Vital Animal Health,Agriculture,NaN
2,minerals and supplements,aviboost nutri block,1kg,1.0,1,Vital Animal Health,ANP062A,2676,1,Vital Animal Health,Agriculture,NaN
3,minerals and supplements,aviboost spectrum,1lt,1.0,1,Vital Animal Health,ANP063A,2677,1,Vital Animal Health,Agriculture,NaN
4,minerals and supplements,aviboost poultry tonic,1lt,1.0,1,Vital Animal Health,ANP003A,2678,1,Vital Animal Health,Agriculture,NaN


In [10]:
category_df.drop_duplicates(subset=['product_name'], keep='first', inplace=True)
category_df.shape

(22595, 3)

In [11]:
category_df = category_df.merge(iprocure_prods_df[['product_name', 'Category', 'Sub category']], on='product_name', how='left')
category_df.drop_duplicates(subset=['product_name'], keep='first', inplace=True)
category_df.head()

,product_name,category_name,sub_category,Category,Sub category
0,gradometer 480 ec,local,NaN,hygiene,NaN
1,magazine east african,locall,NaN,NaN,NaN
2,vectoclor,veterinary products,NaN,veterinary products,NaN
3,chastity perfume men,perfume,NaN,NaN,NaN
4,pannar 15m sunflower,seeds,NaN,NaN,NaN


In [12]:
category_df.shape

(22595, 5)

In [13]:
category_df.isna().sum()

product_name         0
category_name        0
sub_category     22396
Category         19227
Sub category     21438
dtype: int64

In [16]:
# category_df['Category'].value_counts()

### Cleaning category column

In [17]:
category_df['Category'] = np.where(category_df['Category'].isna(), category_df['category_name'], category_df['Category'])
category_df.head()

,product_name,category_name,sub_category,Category,Sub category
0,gradometer 480 ec,local,NaN,hygiene,NaN
1,magazine east african,locall,NaN,locall,NaN
2,vectoclor,veterinary products,NaN,veterinary products,NaN
3,chastity perfume men,perfume,NaN,perfume,NaN
4,pannar 15m sunflower,seeds,NaN,seeds,NaN


In [18]:
category_df.isna().sum()

product_name         0
category_name        0
sub_category     22396
Category             0
Sub category     21438
dtype: int64

In [19]:
# cleaning category column
categories = iprocure_prods_df['Category'].unique().tolist()

wrong_categories_df = category_df[~category_df['Category'].isin(categories)]
wrong_categories_df = wrong_categories_df.drop_duplicates(subset='Category', keep='first')

# cleanup function
def compare(i):
    comparison = {}
    if isinstance(i, str):
       comparison.update({i: get_close_matches(i, categories, n=1, cutoff=0.1)})
    category = list(comparison.keys()) if comparison else None
    match = []
    score = []
    if comparison:
       for key, value in comparison.items():
           if value:
              match.append(value[0])
              score.append(round(SequenceMatcher(None, i, value[0]).ratio(), 2))
           else:
              match.append(None)
              score.append(None)
    else:
       match.append(None)
       score.append(None)
              
    return pd.Series([category, match, score], index = ['category', 'match', 'score'])

cleaned_categories_df = pd.DataFrame()
cleaned_categories_df[['category', 'match', 'score']] = wrong_categories_df['Category'].apply(lambda x: compare(x))
cleaned_categories_df = cleaned_categories_df.applymap(lambda x: x[0] if x else '')
cleaned_categories_df.head()

,category,match,score
1,locall,immunologicals,0.50
3,perfume,equipment,0.38
5,motor rewinding ( service),ear and nose medicines,0.25
7,selective herbicide,hybrid/electric,0.47
10,maize,brake,0.40


In [20]:
# replacing incorrect category names
category_matches_df = cleaned_categories_df[cleaned_categories_df['score'] >= 0.7]
category_matches_df = category_matches_df.rename(columns={'category': 'Category'})
category_matches_df.head()

,Category,match,score
47,agrochemical,agrochemicals,0.96
61,stationary,sanitation,0.70
95,fertilizer.,fertilizers,0.91
172,folia fertilizer,fertilizers,0.74
180,vitamin supplements,minerals and supplements,0.70


In [21]:
category_df = category_df.merge(category_matches_df[['Category', 'match']], how='left', on='Category')
category_df['match'] = np.where(category_df['match'].isna(), category_df['Category'], category_df['match'])
category_df = category_df.drop(['category_name', 'Category'], axis = 1).\
    rename(columns={'match': 'category_name'})
category_df.head()

,product_name,sub_category,Sub category,category_name
0,gradometer 480 ec,NaN,NaN,hygiene
1,magazine east african,NaN,NaN,locall
2,vectoclor,NaN,NaN,veterinary products
3,chastity perfume men,NaN,NaN,perfume
4,pannar 15m sunflower,NaN,NaN,seeds


In [22]:
category_df.isna().sum()

product_name         0
sub_category     22396
Sub category     21438
category_name        0
dtype: int64

### Cleaning sub-category column

In [23]:
category_df['Sub category'] = np.where(category_df['Sub category'].isna(), category_df['sub_category'], category_df['Sub category'])
category_df.head()

,product_name,sub_category,Sub category,category_name
0,gradometer 480 ec,NaN,NaN,hygiene
1,magazine east african,NaN,NaN,locall
2,vectoclor,NaN,NaN,veterinary products
3,chastity perfume men,NaN,NaN,perfume
4,pannar 15m sunflower,NaN,NaN,seeds


In [24]:
category_df.isna().sum()

product_name         0
sub_category     22396
Sub category     21416
category_name        0
dtype: int64

In [27]:
category_df['Sub category'].unique()

array([nan, 'Herbicide', 'Vegetable Seeds', 'Insecticide',
       'Foliar Fertilizer', 'Fungicide', 'Cereal Seeds', 'Hygiene',
       'Powder', 'Block'], dtype=object)

In [26]:
category_df['Sub category'] = category_df['Sub category'].replace('Foliar Fertilizers', 'Foliar Fertilizer')

In [28]:
category_df.shape

(22595, 4)

In [29]:
category_df = category_df.drop('sub_category', axis = 1).\
    rename(columns={'Sub category': 'sub_category'})
category_df.head()

,product_name,sub_category,category_name
0,gradometer 480 ec,NaN,hygiene
1,magazine east african,NaN,locall
2,vectoclor,NaN,veterinary products
3,chastity perfume men,NaN,perfume
4,pannar 15m sunflower,NaN,seeds


In [30]:
# category_df = category_df.drop_duplicates(subset=['product_name'], keep='last').reset_index(drop=True)
category_df[category_df.duplicated(subset=['product_name'])]

,product_name,sub_category,category_name


In [31]:
category_df.isna().sum() # ---> Old

product_name         0
sub_category     21416
category_name        0
dtype: int64

In [32]:
category_df.isna().sum() # ---> New

product_name         0
sub_category     21416
category_name        0
dtype: int64

In [29]:
# agrochem_df = category_df[category_df['category_name'] == 'agrochemicals']
# agrochem_df.drop_duplicates(subset=['product_name'], keep='first', inplace=True)

# x = agrochem_df[agrochem_df['sub_category'].isna()]
# x.to_csv('Agrochemicals_without_subcategory.xlsx', index=False)

In [33]:
category_df

,product_name,sub_category,category_name
0,gradometer 480 ec,NaN,hygiene
1,magazine east african,NaN,locall
2,vectoclor,NaN,veterinary products
3,chastity perfume men,NaN,perfume
4,pannar 15m sunflower,NaN,seeds
...,...,...,...
22590,ultramix maziwa bucket,NaN,minerals and supplements
22591,vitapoultry 100g,NaN,minerals and supplements
22592,vetpower milkreplacer,NaN,minerals and supplements
22593,vetlink multivitamin,NaN,minerals and supplements


In [34]:
sub_df = pd.read_csv('/home/natasha/Documents/Iprocure/Sales-Data-Cleanup/data/subcategories.csv')
sub_df['product_name'] = sub_df['product_name'].apply(lambda x: x.lower().strip())
sub_df.head()

,product_name,sub_category
0,lancer 130 sc,Insecticide
1,nature guard 525 wdg,Fungicide
2,vanguisher 26% wdg,Fungicide
3,elglysate 480sl,Herbicide
4,alwin gold 500mls,Fungicide


In [35]:
category_df = category_df.merge(sub_df, on='product_name', how='left')
category_df.isna().sum()

product_name          0
sub_category_x    21416
category_name         0
sub_category_y    22577
dtype: int64

In [36]:
category_df

,product_name,sub_category_x,category_name,sub_category_y
0,gradometer 480 ec,NaN,hygiene,NaN
1,magazine east african,NaN,locall,NaN
2,vectoclor,NaN,veterinary products,NaN
3,chastity perfume men,NaN,perfume,NaN
4,pannar 15m sunflower,NaN,seeds,NaN
...,...,...,...,...
22590,ultramix maziwa bucket,NaN,minerals and supplements,NaN
22591,vitapoultry 100g,NaN,minerals and supplements,NaN
22592,vetpower milkreplacer,NaN,minerals and supplements,NaN
22593,vetlink multivitamin,NaN,minerals and supplements,NaN


In [37]:
category_df['sub_category_x'] = np.where(category_df['sub_category_x'].isna(), category_df['sub_category_y'], category_df['sub_category_x'])
category_df.isna().sum()

product_name          0
sub_category_x    21398
category_name         0
sub_category_y    22577
dtype: int64

In [38]:
category_df = category_df.drop('sub_category_y', axis = 1).\
    rename(columns={'sub_category_x': 'sub_category'})
category_df

,product_name,sub_category,category_name
0,gradometer 480 ec,NaN,hygiene
1,magazine east african,NaN,locall
2,vectoclor,NaN,veterinary products
3,chastity perfume men,NaN,perfume
4,pannar 15m sunflower,NaN,seeds
...,...,...,...
22590,ultramix maziwa bucket,NaN,minerals and supplements
22591,vitapoultry 100g,NaN,minerals and supplements
22592,vetpower milkreplacer,NaN,minerals and supplements
22593,vetlink multivitamin,NaN,minerals and supplements


In [39]:
category_df.isna().sum()

product_name         0
sub_category     21398
category_name        0
dtype: int64

In [40]:
category_df.to_csv('clean_categories.csv', index=False)

In [41]:
category_df[category_df['category_name'] == 'air and fuel delivery']

,product_name,sub_category,category_name
878,seal,NaN,air and fuel delivery
6602,17801 air cleaner,NaN,air and fuel delivery
11672,cabin filter 87139,NaN,air and fuel delivery
17747,90915 10001 oil,NaN,air and fuel delivery


In [42]:
# loading category data
df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/data_v2/dirty_category_data.csv')

# dropping duplicates and renaning col
df = df.drop_duplicates(subset=['correct_product_match'], keep='first').reset_index(drop=True)
df.rename(columns={'correct_product_match': 'product_name'},inplace=True)
df.head()

,product_name,category_name,sub_category
0,gradometer 480 ec,Local,NaN
1,magazine east african,locall,NaN
2,vectoclor,VETERINARY PRODUCTS,NaN
3,chastity perfume men,Perfume,NaN
4,pannar 15m sunflower,seeds,NaN


In [43]:
df[['category_name', 'product_name']] = df[['category_name', 'product_name']].applymap(lambda x: str(x).lower())

In [45]:
df[df['category_name'] == 'fungocide']

,product_name,category_name,sub_category
69,tabs cream,fungocide,NaN
